# Домашнее задание 2. По чуть-чуть про pandas, sklearn, kNN

В этом задании вы в миниатюре пройдете цикл решения задачи анализа данных от предобработки данных до построения модели и оценки качества. Вы будете решать задачу кредитного скоринга (вернет клиент кредит или нет) на примере небольшой выборки данных. 

В процессе решения задания вы поближе познакомитесь с библиотеками pandas, sklearn и тем, как работают классы в python, а также немного вспомните numpy, самостоятельно реализовав 1NN (это очень просто, потому что сам алгоритм очень простой).

Все задачи нужно выполнять исключительно самостоятельно! Если что-то непонятно, задавайте вопросы в чате. При обнаружении списывания ноль за задание ставится и тому, кто списал, и тому, у кого списали.

Также во всем задании запрещено использование циклов (исключение - второе бонусное задание)! Все операции необходимо выполнять средствами numpy, pandas и sklearn.

Разделение выборки на обучение и контроль производится случайно (если только вы не указали конкретный random_state), поэтому результаты при разных запусках ноутбука могут получаться разные.

Считывание данных уже сделано:

In [2]:
import numpy as np
import pandas as pd

In [3]:
tab = pd.read_csv("https://raw.githubusercontent.com/nadiinchi/iad2019/master/materials/scoring.csv")

Выведите первые несколько строчек датасета, посмотрите названия колонок:

In [4]:
tab.head()

,account,duration,credit_history,purpose,amount,savings,employment,installment_rate,guarantors,residence,...,job,maintenance_people,telephone,foreign,real_estate,life_insurance,car,housing_rent,sex,target
0,0,18,4,furniture,3780,1,1,3,1,2,...,3,1,1,yes,0,0,1,0,male,1
1,1,24,2,car_new,1246,1,1,4,1,2,...,1,1,0,yes,1,0,0,0,male,0
2,0,15,2,car_used,3029,1,3,2,1,2,...,2,1,0,yes,0,0,1,0,male,1
3,0,36,3,television,4463,1,2,4,1,2,...,3,1,1,yes,0,0,1,0,male,0
4,-1,27,2,television,3416,1,2,3,1,2,...,3,1,0,yes,0,0,1,0,male,1


In [5]:
tab.columns

Index(['account', 'duration', 'credit_history', 'purpose', 'amount', 'savings',
       'employment', 'installment_rate', 'guarantors', 'residence', 'age',
       'credits_number', 'job', 'maintenance_people', 'telephone', 'foreign',
       'real_estate', 'life_insurance', 'car', 'housing_rent', 'sex',
       'target'],
      dtype='object')

Сначала несколько вопросов просто так:
    
 __[0.5 балла]__ Какой процент клиентов моложе 30 лет (колонка age) снимает жилье (колонка housing_rent)? А какой процент клиентов старше 40 лет снимает жилье?

In [6]:
young_house = len(tab[(tab.age < 30) & (tab.housing_rent != 0)])
young = sum(tab.age < 30)

In [7]:
print(str(round(young_house / young * 100, 2)) + '%')

31.6%


In [8]:
elder_house = len(tab[(tab.age > 40) & (tab.housing_rent != 0)])
elder = sum(tab.age > 40)

In [9]:
print(str(round(elder_house / elder * 100, 2)) + '%')

9.68%


 __[0.5 балла]__ У скольких иностранцев (foreign == 'yes') число кредитов (credits_number) больше 2?

In [10]:
foreign_credits = len(tab[(tab.foreign == 'yes') & (tab.credits_number > 2)])

In [11]:
foreign_credits

19

 __[0.5 балла]__ Выведите колонки job, life_insurance и guarantors для клиентов с номерами 105, 158, 306 и 336 одной таблицей (воспользуйтесь индексацией):

In [12]:
tab.loc[[105, 158, 306, 336], ['job', 'life_insurance', 'guarantors']]

,job,life_insurance,guarantors
105,2,1,1
158,2,0,1
306,3,0,0
336,2,1,1


 __[0.5 балла]__ Выведите среднюю сумму кредита (колонка amount) в каждой категории (колонка purpose), воспользовавшись методом groupby:

In [13]:
purpose_amount = tab.groupby('purpose')['amount'].mean()

In [14]:
print(purpose_amount)

purpose
business               4405.098361
car_new                3156.796178
car_used               5690.403509
domestic_appliances    1307.875000
education              3276.258065
furniture              3312.677966
others                 8681.700000
repairs                2792.769231
retraining             1156.000000
television             2554.237762
Name: amount, dtype: float64


Теперь по существу: нужно подготовить данные к построению модели.

 __[0.5 балла]__ Как вы видите, не все данные являются числовыми. С признаком purpose мы работать не будем - удалите его из датафрейма:

In [15]:
tab.drop('purpose', axis=1, inplace=True)

 __[0.5 балла]__ Необходимо перекодировать оставшиеся строковые значения в числа (закодировать пол female->0 и male->1 и колонку foreign числами yes->1 и no->0). Сделайте это с помощью pandas (без циклов, конечно же):

In [16]:
tab['sex'] = tab['sex'].map({'female': 0, 'male': 1})

In [17]:
tab['foreign'] = tab['foreign'].map({'no': 0, 'yes': 1})

 __[0.5 балла]__ Целевая переменная хранится в колонке target (последняя колонка). Являются ли классы сбалансированными (одинаковое число 0 и 1)? Воспользуйтесь методом [value_counts](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.value_counts.html).

In [18]:
if tab['target'].value_counts()[1] == tab['target'].value_counts()[1]: print('Yes')

Yes


Выведем статистику датасета:

In [637]:
tab.describe()

,account,duration,credit_history,amount,savings,employment,installment_rate,guarantors,residence,age,...,job,maintenance_people,telephone,foreign,real_estate,life_insurance,car,housing_rent,sex,target
count,600.000000,600.000000,600.000000,600.000000,600.000000,600.000000,600.000000,600.000000,600.000000,600.000000,...,600.000000,600.000000,600.000000,600.000000,600.000000,600.000000,600.000000,600.000000,600.000000,600.000000
mean,0.083333,21.878333,2.410000,3470.491667,1.200000,2.348333,2.988333,0.901667,2.831667,35.181667,...,1.906667,1.150000,0.391667,0.965000,0.268333,0.205000,0.343333,0.186667,0.598333,0.500000
std,0.915306,12.749161,1.098747,3090.052187,0.924531,1.208069,1.131517,0.298013,1.103945,11.416182,...,0.662334,0.357369,0.488530,0.183933,0.443462,0.404038,0.475218,0.389969,0.490644,0.500417
min,-1.000000,4.000000,0.000000,250.000000,0.000000,0.000000,1.000000,0.000000,1.000000,19.000000,...,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,-1.000000,12.000000,2.000000,1363.250000,1.000000,2.000000,2.000000,1.000000,2.000000,26.000000,...,2.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,18.000000,2.000000,2347.000000,1.000000,2.000000,3.000000,1.000000,3.000000,33.000000,...,2.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.500000
75%,1.000000,27.000000,4.000000,4318.750000,1.000000,3.000000,4.000000,1.000000,4.000000,41.000000,...,2.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,0.000000,1.000000,1.000000
max,2.000000,72.000000,4.000000,18424.000000,4.000000,4.000000,4.000000,1.000000,4.000000,75.000000,...,3.000000,2.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [638]:
tab['credits_number'].describe()

count    600.000000
mean       1.366667
std        0.565174
min        1.000000
25%        1.000000
50%        1.000000
75%        2.000000
max        4.000000
Name: credits_number, dtype: float64

 __[0.5 балла]__ Проанализируйте таблицу. Есть ли признаки с пропущенными значениями? <br><br>
 Можно заметить, что колонка __count__ у всех столбцов равна 600, что значит, что значений _NaN_ в таблице нет. Значит, стоит искать "странные" значения.<br>
 Если смотреть на столбец __account__, то там минимальное значение равно _-1_. Возможно, это и является "символом" пропущенного значения.

In [652]:
print(tab.count(), '\n')
print(tab['account'].describe())

account               600
duration              600
credit_history        600
amount                600
savings               600
employment            600
installment_rate      600
guarantors            600
residence             600
age                   600
credits_number        600
job                   600
maintenance_people    600
telephone             600
foreign               600
real_estate           600
life_insurance        600
car                   600
housing_rent          600
sex                   600
target                600
dtype: int64 

count    600.000000
mean       0.083333
std        0.915306
min       -1.000000
25%       -1.000000
50%        0.000000
75%        1.000000
max        2.000000
Name: account, dtype: float64


 __[0.5 балла]__ Запишите в переменную X матрицу объекты-признаки (выделите все колонки признаков, кроме target), а в переменную y - вектор правильных ответов (колонка target). К обоим объектам припишите .values, чтобы "достать" numpy-матрицу из датафрейма: далее мы везде будем работать с numpy-матрицами.

In [28]:
X = tab[tab.columns[:-1]].values
y = tab[tab.columns[-1]].values

 __[0.5 балла]__ С помощью sklearn-функции train_test_split разделите данные на обучение и контроль (пропорцию оставьте по умолчанию). Ячейка ?train_test_split покажет документацию функции (с примерами использования), также можно загуглить документацию.

In [54]:
from sklearn.model_selection import train_test_split

In [32]:
?train_test_split

In [94]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

Выведите форму (shape) обучающей и контрольной выборок:

In [259]:
X_train.shape, X_test.shape,y_train.shape, y_test.shape

((450, 20), (150, 20), (450,), (150,))

 __[0.5 балла]__ Как вы видели выше, все колонки имеют разный масштаб, это будет мешать нам при запуске метрического классификатора. Отнормируйте данные с помощью StandardScaler (интерфейс такой же, как мы изучали на семинаре):

In [96]:
from sklearn.preprocessing import StandardScaler

In [97]:
scaler = StandardScaler()

In [98]:
scalerModel = scaler.fit_transform(X_train, y_train)

In [100]:
scalerModel = scaler.transform(X_test)

 __[0.5 балла]__ Теперь обучите классификатор 1-NN из sklearn на обучающей выборке, сделайте предсказания отдельно для обучающей и для контрольной выборки, на каждой выборке посчитайте точность (accuracy) с помощью accuracy_score. При создании объекта KNeighborsClassifier необходимо указать число соседей 1 (по умолчанию оно больше). 

Интерфейс KNeighborsClassifier такой же, как мы изучали на семинаре, а accuracy-score принимает на вход вектор предсказаний и вектор правильных ответов.

In [101]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

In [236]:
knc = KNeighborsClassifier(n_neighbors=1)
knc.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=1)

In [237]:
preds_train = knc.predict(X_train)
preds_test = knc.predict(X_test)

In [238]:
accuracy_score(y_train, preds_train)

1.0

In [239]:
accuracy_score(y_test, preds_test)

0.54

 __[4 балла,  включая my_accuracy и тестирование реализации]__ Наконец, реализуйте свой 1NN, воспользовавшись шаблоном ниже. Ваш класс должен уметь работать в двух режимах: в евклидовой метрикой и с манхэттанской метрикой (формулы метрик ниже). Напоминаем принцип работы 1NN: чтобы сделать предсказание для объекта, он выбирает ближайший объект из обучения и возвращает такой же класс, как у него.
* в методе \__init\__ необходимо запомнить, с какой метрикой мы работаем
* в методе fit нужно запомнить выборку - больше при обучении kNN ничего делать не нужно
* в методе predict нужно сделать три шага:
    * сначала построить матрицу расстояний между контрольными и обучающими объектами. Для этого можно использовать функцию [cdist](https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.distance.cdist.html) (прочитайте ее документацию).
    * затем взять argmin по строкам - получится вектор индексов ближайших обучающих объектов для каждого контрольного объекта
    * выбрать компоненты y_train, отвечающие компонентам вектора индексов. Иными словами, вернуть те же метки классов, что у ближайших обучающих объектов.
    
Формулы метрик (расстояний):
* евклидова: $$\rho(x, y) = \sqrt{\sum_{i=1}^d (x_i - y_i)^2}$$
* манхэттанская: $$\rho(x, y) = \sum_{i=1}^d |x_i - y_i|$$

Модуль в numpy вычисляется как np.abs.

In [115]:
from scipy.spatial.distance import cdist

In [468]:
class MyKNN:
    def __init__(self, metric='euclid'):
        self.metric_ = metric
        
    def fit(self, X_tr, y_tr):
        self.X = X_tr
        self.y = y_tr
        
    def predict(self, X_te):
        if self.metric_ == 'euclid':
            matrix = cdist(self.X, X_te, 'euclid')
        elif self.metric_ == 'cityblock':
            matrix = cdist(self.X, X_te, 'cityblock')
        vector = np.argmin(matrix, axis = 0)
        nearest_label = self.y[vector]
        return nearest_label

Реализуйте также функцию вычисления точности - доли правильных ответов:

In [306]:
def my_accuracy(y_pred, y_test):
    return (y_pred == y_test).mean()

Пришло время запустить ваш классификатор и вычислить точность с помощью вашей функции. Обучите классификатор, сделайте предсказания для обучающей и контрольной выборок и посчитайте точность в обоих случаях. Сделайте это два раза - ддя евклидовой и манхэттанской метрик. Для евклидовой метрики цифры должны получиться точно такие же, как для реализации sklearn.

In [630]:
my_knn = MyKNN('euclid')
my_knn.fit(X_train, y_train)
my_preds_train_e = my_knn.predict(X_train)
my_preds_test_e = my_knn.predict(X_test)

In [631]:
my_accuracy(my_preds_train_e, y_train)

1.0

In [632]:
my_accuracy(my_preds_test_e, y_test)

0.54

In [605]:
my_knn = MyKNN('cityblock')
my_knn.fit(X_train, y_train)
my_preds_train_c = my_knn.predict(X_train)
my_preds_test_c = my_knn.predict(X_test)

In [473]:
my_accuracy(my_preds_train_c, y_train)

1.0

In [606]:
my_accuracy(my_preds_test_c, y_test)

0.5866666666666667

### Бонусное задание: реализация kNN
Добавьте в ваш KNN параметр k (число соседей) и реализуйте k-NN, который возвращает самый популярный класс среди k ближайших соседей объекта. Если число ближайших объектов для разных классов совпадает, возвращайте класс с меньшим номером. За реализацию, работающую для любого числа классов - 3 балла (посмотрите функции np.bincount, np.apply_along_axis (к сожалению, bincount не применяется построчно), np.argpartition), за реализацию, работающую только для двух классов - 2 балла. Не забудьте протестировать свой алгоритм :) В этом задании циклы использовать нельзя!


In [623]:
class MyKNN_n:
    def __init__(self, metric='euclid', number=3):
        self.metric_ = metric
        if number != 3:
            self.number_ = number
        else:
            self.number_ = 3
        
    def fit(self, X_tr, y_tr):
        self.X = X_tr
        self.y = y_tr
        
    def predict(self, X_te):
        classes = len(np.unique(self.y))
        if self.metric_ == 'euclid':
            n_matrix = cdist(self.X, X_te, 'euclid')
        elif self.metric_ == 'cityblock':
            n_matrix = cdist(self.X, X_te, 'cityblock')
        nearests = np.argpartition(n_matrix, (0, self.number_))[:self.number_]
        n_nearest_labels = np.transpose(np.array(self.y[nearests]))
        nearest_label_count = np.apply_along_axis(np.bincount, 1, n_nearest_labels, minlength=classes)
        n_nearest_label = np.apply_along_axis(np.argmax, 1, nearest_label_count)
        return n_nearest_label